In [1]:
%%writefile submission.py
import base64
import pickle
import zlib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.container import Sequential
from torch.distributions.categorical import Categorical
from kaggle_environments.envs.hungry_geese.hungry_geese import Action


class FeaturesExtractor(nn.Module):    
    def __init__(self, features_dim=64*7*11):
        super().__init__()
        n_channels = 11
        self.conv1 = nn.Conv2d(n_channels, 32, (3, 3))
        self.conv2 = nn.Conv2d(32, 64, (3, 3))
        self.flatten = nn.Flatten(start_dim=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flatten(x)
        return x


class MlpExtractor(nn.Module):
    def __init__(self):
        super(MlpExtractor, self).__init__()
        self.shared_net = Sequential(
            nn.Linear(64*7*11, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )
        self.policy_net = Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.value_net = Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.shared_net(x)
        p = self.policy_net(x)
        v = self.value_net(x)
        return p, v


class ActorCriticPolicy(nn.Module):
    def __init__(self):
        super(ActorCriticPolicy, self).__init__()
        self.features_extractor = FeaturesExtractor()
        self.mlp_extractor = MlpExtractor()
        self.action_net = nn.Linear(in_features=64, out_features=4, bias=True)
        self.value_net = nn.Linear(in_features=64, out_features=1, bias=True)

    def forward(self, x):
        x = self.features_extractor(x)
        p, v = self.mlp_extractor(x)
        p = self.action_net(p)
        v = self.value_net(v)
        return p, v


state_dict = _STATE_DICT_

state_dict = pickle.loads(zlib.decompress(base64.b64decode(state_dict)))
model = ActorCriticPolicy()
model.load_state_dict(state_dict)
model.eval()

obs_prev = None
act_prev = None


# Modified from https://www.kaggle.com/yuricat/smart-geese-trained-by-reinforcement-learning
def process_obs(obs):
    global obs_prev
    obs_index = obs.index

    b = np.zeros((7 * 11, 11), dtype=np.uint8)
    b[:, -1] = 255  # empty cells

    for p, pos_list in enumerate(obs.geese):
        # head position
        for pos in pos_list[:1]:
            b[pos, 0 + (p - obs_index) % 4] = 255
            b[pos, -1] = 0
        # whole position
        for pos in pos_list:
            b[pos, 4 + (p - obs_index) % 4] = 255
            b[pos, -1] = 0

    # previous head position
    if obs_prev is not None:
        for pos in obs_prev.geese[obs_index][:1]:
            b[pos, -3] = 255

    for pos in obs.food:
        b[pos, -2] = 255

    b = b.reshape(7, 11, -1)

    b = np.concatenate([b[:, -2:],
                        b,
                        b[:, :2]], axis=1)
    b = np.concatenate([b[-2:, :],
                        b,
                        b[:2, :]], axis=0)
    return b


def agent(obs, conf):
    global model, obs_prev, act_prev
    obs_backup = obs
    obs = process_obs(obs) / 255.  # normalize
    obs = np.expand_dims(obs.transpose(2, 0, 1), 0)
    obs = torch.from_numpy(obs.astype(np.float32))
    p, v = model(obs)
    p = p.squeeze()
    print(p)
    p = F.softmax(p, dim=0)
    if act_prev is not None:
        act_oppo = (act_prev + 1) % 4 + 1
        p[act_oppo - 1] = 0
        p /= p.sum()
    # action = p.squeeze().argmax().item() + 1
    action = Categorical(p).sample().item() + 1
    obs_prev = obs_backup
    act_prev = action
    return Action(action).name

Overwriting submission.py


In [2]:
import os
import glob
import base64
import pickle
import zlib
from stable_baselines3 import PPO
from kaggle_environments import make

path = 'models1'
list_of_files = glob.glob(os.path.join(path, '*.zip'))
model_path = max(list_of_files, key=os.path.getmtime)
print(model_path)
model = PPO.load(model_path)
# print(model.policy)

state_dict = model.policy.to('cpu').state_dict()
state_dict = base64.b64encode(zlib.compress(pickle.dumps(state_dict)))

with open('submission.py', 'r') as file:
    src = file.read()
src = src.replace("_STATE_DICT_", f"{state_dict}")
with open('submission.py', 'w') as file:
    file.write(src)

Loading environment football failed: No module named 'gfootball'
models1/model_400000_steps.zip


In [3]:
env = make("hungry_geese", debug=True)
# env.run(["examples/risk_averse.py", "examples/mighty_boiler_goose.py", "examples/simple_bfs.py", "submission.py"])
env.run(["submission.py", "submission.py", "submission.py", "submission.py"])
env.render(mode="ipython", width=700, height=550)
# white, blue, green, red

[-2.3223, -2.0704,  0.6497,  1.7333], grad_fn=<SqueezeBackward0>)
tensor([-1.8724, -1.8474,  1.0930,  1.0495], grad_fn=<SqueezeBackward0>)
tensor([-2.5583, -2.5055,  3.4900, -0.6578], grad_fn=<SqueezeBackward0>)
tensor([-2.0458, -2.1564,  1.2649,  1.1556], grad_fn=<SqueezeBackward0>)
tensor([-2.0562, -2.1114,  1.0618,  1.3155], grad_fn=<SqueezeBackward0>)
tensor([-2.1307, -2.0686, -0.6953,  2.6940], grad_fn=<SqueezeBackward0>)
tensor([-2.5672, -2.5717,  1.3667,  1.5475], grad_fn=<SqueezeBackward0>)
tensor([-2.3695, -2.4303,  1.4522,  1.3056], grad_fn=<SqueezeBackward0>)
tensor([-2.2506, -2.1390,  1.8935,  0.6828], grad_fn=<SqueezeBackward0>)
tensor([-1.8871, -1.8665,  2.8482, -0.7746], grad_fn=<SqueezeBackward0>)
tensor([-2.3466, -2.4179,  1.8869,  0.8935], grad_fn=<SqueezeBackward0>)
tensor([-2.2357, -2.1034,  1.0477,  1.3554], grad_fn=<SqueezeBackward0>)
tensor([-2.7399, -2.5649,  1.1881,  1.7493], grad_fn=<SqueezeBackward0>)
tensor([-2.5251, -2.7115,  1.3139,  1.6687], grad_fn=<Sque

In [4]:
!kaggle competitions submit -c hungry-geese -f submission.py -m "PPO MlpPolicy(ReLU) self-play"

100%|██████████████████████████████████████| 13.0M/13.0M [00:06<00:00, 2.08MB/s]
Successfully submitted to Hungry Geese